# Fine-Tuning of OpenAI detector

In [3]:
import json
import pandas as pd
import numpy as np
import torch
import tqdm 
import torch.nn.functional as F
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

c:\Users\bobae\anaconda3\envs\ai_detector\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

### Load json data file and convert to dataframe

In [5]:
# human data

# loads a tofel dataset
with open('../dataset/human/tofel.json', "r") as f:
    h_tofel_dataset = json.load(f)

# loads an arxiv dataset
with open('../dataset/human/arxiv.json', "r") as f:
    h_arxiv_dataset = json.load(f)

# loads student essay
with open('../dataset/human/student_essay.json', "r") as f:
    h_essay_dataset = json.load(f)

# loads student computer essay 
with open('../dataset/human/student_cs_essay.json', "r") as f:
    h_essay_cs_dataset = json.load(f)

In [10]:
# gpt data

# loads a tofel dataset
with open('../dataset/ai/gpt2medium_tofel.json', "r") as f:
    gpt_tofel_dataset = json.load(f)

# loads an arxiv dataset
with open('../dataset/ai/gpt2medium_arxiv.json', "r") as f:
    gpt_arxiv_dataset = json.load(f)

# loads student essay
with open('../dataset/ai/gpt2medium_essay.json', "r") as f:
    gpt_essay_dataset = json.load(f)

# loads student computer essay 
with open('../dataset/ai/gpt2medium_essay_cs.json', "r") as f:
    gpt_essay_cs_dataset = json.load(f)
    
# loads a tofel dataset    
with open('../dataset/ai/gpt35_tofel.json', "r") as f:
    gpt_35_tofel_dataset = json.load(f)
    
# loads student computer essay
with open('../dataset/ai/gpt35_essay_cs.json', "r") as f:
    gpt_35_essay_cs_dataset = json.load(f)

In [13]:
h_dataset = []
for i in [h_tofel_dataset, h_arxiv_dataset, h_essay_dataset, h_essay_cs_dataset]:
    h_dataset.extend(i)

len(h_dataset)

2478

In [11]:
gpt_dataset = []
for i in [gpt_tofel_dataset, gpt_arxiv_dataset, gpt_essay_dataset, gpt_essay_cs_dataset,
          gpt_35_tofel_dataset, gpt_35_essay_cs_dataset]:
    gpt_dataset.extend(i)

len(gpt_dataset)

2956

In [14]:
dct = {
    "text": [item['input'] for item in h_dataset] + [item['input'] for item in gpt_dataset],
    "label": [item['label'] for item in h_dataset] + [item['label'] for item in gpt_dataset],
}

df = pd.DataFrame(dct) 

def label_to_numeric(value):
    if value == "human": 
        return 1
    else:
        return 0
    
df['target'] = df['label'].apply(lambda x: label_to_numeric(x))

### Preparing the Dataset and Dataloader

In [8]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = self.data.text
        self.target = self.data.target
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            pad_to_max_length=True,
            max_length=self.max_len,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[index], dtype=torch.float)
        }

In [9]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/roberta-base-openai-detector")
model = AutoModelForSequenceClassification.from_pretrained("openai-community/roberta-base-openai-detector")

Some weights of the model checkpoint at openai-community/roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [11]:
train_size = 0.8
train_data=df.sample(frac=train_size, random_state=42)
test_data=df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (4956, 3)
TRAIN Dataset: (3965, 3)
TEST Dataset: (991, 3)


In [12]:
train_params = {'batch_size': 16, 'shuffle': True, 'num_workers': 0}
test_params = {'batch_size': 16, 'shuffle': True, 'num_workers': 0}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

### Fine Tuning the Model

In [13]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [14]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [15]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
def train(epoch, model):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model = model.to(device)
    model.train()
    
    # for i, data in tqdm(enumerate(training_loader, 0), total=len(training_loader)):
    # for _, data in tqdm(enumerate(training_loader, 0)):
    for i, data in enumerate(iter(training_loader)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs.logits, targets)

        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.logits, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if i%100==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")
            print("=="*50)

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [16]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch, model)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/bobaebak/anaconda3/envs/ai_detector/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 0.17000845074653625
Training Accuracy per 5000 steps: 93.75
Training Loss per 5000 steps: 0.03668684901503643
Training Accuracy per 5000 steps: 98.9480198019802
Training Loss per 5000 steps: 0.023941905792010252
Training Accuracy per 5000 steps: 99.25373134328358
The Total Accuracy for Epoch 0: 99.39470365699874
Training Loss Epoch: 0.01973094394311521
Training Accuracy Epoch: 99.39470365699874


In [25]:
def valid(model, testing_loader):
    model = model.to(device)
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        # for _, data in tqdm(enumerate(testing_loader, 0)):
        for i, data in enumerate(iter(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['target'].to(device, dtype = torch.long)
            
            outputs = model(ids, mask, token_type_ids)
            loss = loss_function(outputs.logits, targets)
            
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.logits, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [26]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

KeyboardInterrupt: 

In [39]:
model = model.to(device)
model.eval()

for i, data in enumerate(iter(testing_loader)):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
    targets = data['target']
    
    outputs = model(ids, mask, token_type_ids)
    logits = outputs.logits
    prob = F.softmax(logits, dim=-1)[:, :].detach().cpu().numpy()
    # 0: fake, 1: real
    for _, a in enumerate(prob):
        print(targets[_], a)        
    break

/Users/bobaebak/anaconda3/envs/ai_detector/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor(0.) [9.9980313e-01 1.9692969e-04]
tensor(0.) [9.9980086e-01 1.9911495e-04]
tensor(0.) [9.998054e-01 1.946893e-04]
tensor(0.) [9.9980313e-01 1.9692969e-04]
tensor(1.) [0.00323358 0.99676645]
tensor(0.) [9.9979335e-01 2.0667377e-04]
tensor(1.) [2.0647602e-04 9.9979359e-01]
tensor(0.) [9.9980313e-01 1.9692969e-04]
tensor(1.) [1.847777e-04 9.998153e-01]
tensor(1.) [2.2683069e-04 9.9977320e-01]
tensor(1.) [2.1246231e-04 9.9978751e-01]
tensor(1.) [0.00831866 0.9916814 ]
tensor(1.) [2.248533e-04 9.997751e-01]
tensor(1.) [1.8204887e-04 9.9981803e-01]
tensor(0.) [9.9980408e-01 1.9599737e-04]
tensor(1.) [1.9227664e-04 9.9980778e-01]


In [17]:
# check the model

for item in h_tofel_dataset:
    test = item['input']
    encoded_inputs = tokenizer.encode_plus(
        test,   # Tokenize the sentence.
        None,   # Prepend the `[CLS]` token to the start.
        add_special_tokens=True,
        pad_to_max_length=True,
        max_length=512,
        return_token_type_ids=True,
        truncation=True,
        return_tensors="pt",
    )

    encoded_inputs = encoded_inputs.to(device)

    output = model(
        encoded_inputs.input_ids, 
        encoded_inputs.attention_mask, 
        encoded_inputs.token_type_ids,
    )

    logits = output.logits
    prob = F.softmax(logits, dim=-1)[:, :].detach().cpu().numpy().squeeze()
    print({"Fake": prob[0], "Real": prob[1]})



{'Fake': 0.0004347022, 'Real': 0.9995653}
{'Fake': 0.00055805565, 'Real': 0.9994419}
{'Fake': 0.00038633827, 'Real': 0.9996137}
{'Fake': 0.0010733912, 'Real': 0.9989266}
{'Fake': 0.00028868736, 'Real': 0.99971133}
{'Fake': 0.0004142776, 'Real': 0.9995857}
{'Fake': 0.00023252562, 'Real': 0.9997675}
{'Fake': 0.00033761762, 'Real': 0.9996624}
{'Fake': 0.000558327, 'Real': 0.9994417}
{'Fake': 0.0007697495, 'Real': 0.99923027}
{'Fake': 0.00022874387, 'Real': 0.9997713}
{'Fake': 0.0003081829, 'Real': 0.99969184}
{'Fake': 0.00021044456, 'Real': 0.99978954}
{'Fake': 0.000567336, 'Real': 0.9994326}
{'Fake': 0.0011065092, 'Real': 0.99889356}
{'Fake': 0.00071633206, 'Real': 0.99928373}
{'Fake': 0.0002635781, 'Real': 0.99973637}
{'Fake': 0.000228002, 'Real': 0.999772}
{'Fake': 0.001737065, 'Real': 0.99826294}
{'Fake': 0.00028268207, 'Real': 0.9997173}
{'Fake': 0.00028259447, 'Real': 0.9997174}
{'Fake': 0.0015291135, 'Real': 0.99847084}
{'Fake': 0.005053571, 'Real': 0.9949464}
{'Fake': 0.0002841300

### save the model

In [ ]:
# output_model_file = 'pytorch_roberta_sentiment.bin'
# output_vocab_file = './'

# model_to_save = model
# torch.save(model_to_save, output_model_file)
# tokenizer.save_vocabulary(output_vocab_file)

# print('All files saved')
# print('This tutorial is completed')

In [42]:
torch.save(model, '../models/fine_tune_epoch1.pth')

# Load the saved model
loaded_model = torch.load('../models/fine_tune_epoch1.pth')